### USE PYG ENVIRONMENT!!!!

In [1]:
# !pip install torch


In [2]:
# Install required packages
import os, torch

# os.environ['TORCH'] = torch.__version__
# !pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git

# !pip install sentence_transformers
# !pip3 install fuzzywuzzy[speedup]
# !pip install captum
# !pip install torch-sparse
# !pip install torch-scatter 


In [ ]:
### load graph data 
import pickle

with open('../data/graphs/graph.pkl', 'rb') as f:
    data = pickle.load(f)


In [4]:
### create edge sampling loader
from torch_geometric.data import DataLoader
import torch_geometric.transforms as T
import numpy as np

train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1)(data)


In [20]:
from torch_geometric.loader import LinkNeighborLoader

train_loader = LinkNeighborLoader(
    train_data,
    # Sample 30 neighbors for each node for 2 iterations
    num_neighbors=[10] * 2,
    # Use a batch size of 128 for sampling training nodes
    batch_size=128,
)

val_loader = LinkNeighborLoader(
    val_data,
    num_neighbors=[10] * 2,
    batch_size=128,
)

test_loader = LinkNeighborLoader(
    test_data,
    num_neighbors=[10] * 2,
    batch_size=128,
)
### create model
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(data.num_features, 1)
        self.conv2 = GCNConv(1, 1)
        self.lin = Linear(data.num_features, 1)

    def forward(self, x, edge_index, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return self.lin(x)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### train model
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.mse_loss(out, data.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_nodes

    return total_loss / len(train_loader)

### train model
for epoch in range(1, 101):
    loss = train()
    print('Epoch: {:03d}, Loss: {:.4f}'.format(epoch, loss))
